In [2]:
import requests
import json

# Define pulls from api

In [5]:
# gateway list
gateway_list_api = requests.get("http://localhost:8080/api/v1/structure/gateway/list")

# specific gateway
g_id = None
gateway_api = requests.get(f"http://localhost:8080/api/v1/structure/tag/list/{g_id}")

In [6]:
gateway_list_api.status_code

200

## Gateway list

In [8]:
gateway_list: dict = json.loads(gateway_list_api.text)
gateway_list.keys()

dict_keys(['gateways'])

In [11]:
gateway_list["gateways"]

[{'network_segment': 0,
  'last_contact': '2023-09-03T11:34:27.3154321Z',
  'online': True,
  'ip_address': '10.0.0.191',
  'id': '31302e302e302e313931d41d8cd98f00b204e9800998ecf8427e'},
 {'network_segment': 2,
  'last_contact': '2023-09-03T11:34:27.3154492Z',
  'online': True,
  'ip_address': '10.0.2.250',
  'id': '31302e302e322e323530d41d8cd98f00b204e9800998ecf8427e'},
 {'network_segment': 2,
  'last_contact': '2023-09-03T11:34:27.422962Z',
  'online': True,
  'ip_address': '10.0.2.242',
  'id': '31302e302e322e323432d41d8cd98f00b204e9800998ecf8427e'},
 {'network_segment': 0,
  'last_contact': '2023-09-03T11:34:27.6242798Z',
  'online': True,
  'ip_address': '10.0.0.106',
  'id': '31302e302e302e313036d41d8cd98f00b204e9800998ecf8427e'},
 {'network_segment': 2,
  'last_contact': '2023-09-03T11:34:27.8395145Z',
  'online': True,
  'ip_address': '10.0.2.9',
  'id': '31302e302e322e39d41d8cd98f00b204e9800998ecf8427e'},
 {'network_segment': 2,
  'last_contact': '2023-09-03T11:34:27.8757935Z'

In [12]:
gateways: list = gateway_list["gateways"]
len(gateways)

20

# Tags

In [28]:
def get_tag_data(g_id:str) -> list[dict]:
    gateway_api = requests.get(f"http://localhost:8080/api/v1/structure/tag/list/{g_id}")
    return json.loads(gateway_api.text)

In [30]:
tag_data = []
for gateway in gateways:
    tag_data.extend(get_tag_data(gateway["id"]))
tag_data[:3]

[{'sensors': [],
  'address': 'f5:15:c3:50:51:b8',
  'name': 'ruuvi_51b8',
  'last_contact': '2023-09-03T12:05:26.4345421Z',
  'online': True},
 {'sensors': [],
  'address': '36:38:c9:66:bb:5f',
  'name': 'ruuvi_bb5f',
  'last_contact': '2023-09-03T12:05:26.4521632Z',
  'online': True},
 {'sensors': [],
  'address': 'c7:c7:d9:51:f:58',
  'name': 'ruuvi_f58',
  'last_contact': '2023-09-03T12:05:26.469885Z',
  'online': True}]

In [31]:
len(tag_data)

188

### Filter for tags with sensors

In [37]:
from typing import Callable
has_sensor_data:Callable[[Tag], bool] = lambda tag: len(tag["sensors"]) > 0
list(filter(has_sensor_data, tag_data))

[]

# Data model

In [49]:
from dataclasses import dataclass, field
from datetime import datetime

@dataclass
class Tag:
    sensors: list
    address: str
    name: str
    last_contact: str | datetime
    online: bool

@dataclass
class Gateway:
    network_segment: int
    last_contact: str
    online: bool
    ip_address: str
    id: str # id is keyword in python, maybe needs a mapping
    tags: list[Tag] = field(default_factory = list)

# Instantiate data model

## Gateways

In [45]:
# Gateway api
gateway_api = requests.get("http://localhost:8080/api/v1/structure/gateway/list")

gateways: list[Gateway] = []

# if gateway_list_api.status_code == 200
for gateway in json.loads(gateway_api.text)["gateways"]:
    gateways.append(
        Gateway(**gateway)
    )
    
gateways[:3]

[Gateway(network_segment=0, last_contact='2023-09-03T12:51:01.2260019Z', online=True, ip_address='10.0.0.191', id='31302e302e302e313931d41d8cd98f00b204e9800998ecf8427e', tags=[]),
 Gateway(network_segment=2, last_contact='2023-09-03T12:51:01.4392568Z', online=True, ip_address='10.0.2.250', id='31302e302e322e323530d41d8cd98f00b204e9800998ecf8427e', tags=[]),
 Gateway(network_segment=2, last_contact='2023-09-03T12:51:01.5460281Z', online=True, ip_address='10.0.2.242', id='31302e302e322e323432d41d8cd98f00b204e9800998ecf8427e', tags=[])]

## Tags

In [50]:
for gateway in gateways:
    tag_api = requests.get(f"http://localhost:8080/api/v1/structure/tag/list/{gateway.id}")
    tag_list = json.loads(tag_api.text)
    if len(tag_list) > 0:
        for tag in tag_list:
            gateway.tags.append(
                Tag(**tag)
            )
            
gateways[:5]

[Gateway(network_segment=0, last_contact='2023-09-03T12:51:01.2260019Z', online=True, ip_address='10.0.0.191', id='31302e302e302e313931d41d8cd98f00b204e9800998ecf8427e', tags=[Tag(sensors=[], address='b2:62:93:ca:32:a9', name='ruuvi_32a9', last_contact='2023-09-03T12:53:09.6749338Z', online=True), Tag(sensors=[], address='90:90:e4:b0:3f:52', name='ruuvi_3f52', last_contact='2023-09-03T12:53:09.6954427Z', online=True), Tag(sensors=[], address='27:22:a4:f3:6e:9e', name='ruuvi_6e9e', last_contact='2023-09-03T12:53:09.7145584Z', online=True), Tag(sensors=[], address='11:4d:5:6d:5b:6a', name='ruuvi_5b6a', last_contact='2023-09-03T12:53:09.7315777Z', online=True), Tag(sensors=[], address='b7:29:8b:5e:76:a8', name='ruuvi_76a8', last_contact='2023-09-03T12:53:09.7489901Z', online=True), Tag(sensors=[], address='62:f7:c9:89:5b:4f', name='ruuvi_5b4f', last_contact='2023-09-03T12:53:09.7660636Z', online=True), Tag(sensors=[], address='dc:66:2f:c6:a:2c', name='ruuvi_a2c', last_contact='2023-09-03T

In [51]:
len(gateways)

20

In [53]:
has_tags: Callable[[Gateway], bool] = lambda g: len(g.tags) > 0
len(list(filter(has_tags, gateways)))

19

In [55]:
# some test
tags = json.loads(requests.get(f"http://localhost:8080/api/v1/structure/tag/list/{gateways[0].id}").text)
tags

[{'sensors': [],
  'address': 'b2:62:93:ca:32:a9',
  'name': 'ruuvi_32a9',
  'last_contact': '2023-09-03T13:01:21.7345094Z',
  'online': True},
 {'sensors': [],
  'address': '90:90:e4:b0:3f:52',
  'name': 'ruuvi_3f52',
  'last_contact': '2023-09-03T13:01:21.7561314Z',
  'online': True},
 {'sensors': [],
  'address': '27:22:a4:f3:6e:9e',
  'name': 'ruuvi_6e9e',
  'last_contact': '2023-09-03T13:01:21.7767795Z',
  'online': True},
 {'sensors': [],
  'address': '11:4d:5:6d:5b:6a',
  'name': 'ruuvi_5b6a',
  'last_contact': '2023-09-03T13:01:21.7975976Z',
  'online': True},
 {'sensors': [],
  'address': 'b7:29:8b:5e:76:a8',
  'name': 'ruuvi_76a8',
  'last_contact': '2023-09-03T13:01:21.8185254Z',
  'online': True},
 {'sensors': [],
  'address': '62:f7:c9:89:5b:4f',
  'name': 'ruuvi_5b4f',
  'last_contact': '2023-09-03T13:01:21.8367719Z',
  'online': True},
 {'sensors': [],
  'address': 'dc:66:2f:c6:a:2c',
  'name': 'ruuvi_a2c',
  'last_contact': '2023-09-03T13:01:21.8554702Z',
  'online': Tr

In [56]:
has_no_tag: Callable[[Gateway], bool] = lambda g: len(g.tags) == 0
no_tag_gateways = list(filter(has_no_tag, gateways))
no_tag_gateways

[Gateway(network_segment=0, last_contact='2023-09-03T10:52:56.1370102Z', online=False, ip_address='10.0.0.65', id='31302e302e302e3635d41d8cd98f00b204e9800998ecf8427e', tags=[])]

In [57]:
tags = json.loads(requests.get(f"http://localhost:8080/api/v1/structure/tag/list/{no_tag_gateways[0].id}").text)
tags

[]